# Model Inference & Evaluation Pipeline

This script demonstrates how to:
1. Load trained models of different types 
2. Run inference on new data using the loaded model.
3. Compute and plot the overall ROC curve (AUROC).
4. Compute and plot ROC curves (and corresponding AUROC scores) per strata,
    where each strata is defined as the composite of `tissue_type` and `imputed_labels`.

## Imports and Setup

In [ ]:
import sys

from pathlib import Path

import anndata as ad
import numpy as np
import seaborn as sns

# Set matplotlib backend for Jupyter notebooks
%matplotlib inline

project_root = Path.cwd().parent.parent
print(f"Project root: {project_root}")
sys.path.insert(0, str(project_root))

import scxpand

from scxpand.data_util.data_splitter import get_patient_identifiers
from scxpand.data_util.transforms import extract_is_expanded
from scxpand.util.general_util import metrics_dict_to_dataframes
from scxpand.util.plots import plot_roc_curve, plot_roc_curves_per_strata

sns.set_style("whitegrid")

## Configuration


### Model configuration

1. **Local Model**: Set `RESULTS_PATH` to use a model you trained locally (default)
2. **Registry Model**: Set `MODEL_NAME` to use a curated pre-trained model from scXpand 
3. **Direct URL**: Set `MODEL_URL` to use any external model via direct URL

In [ ]:
# Set ONE of the following inference modes:

# === OPTION 1: load model from local path
# RESULTS_PATH = project_root / "results/mlp"
# MODEL_NAME = None
# MODEL_URL = None

# === OPTION 2: load model from registry using the model name (choose from list_pretrained_models())
RESULTS_PATH = None
MODEL_NAME = "pan_cancer_autoencoder"
MODEL_URL = None

# === OPTION 3: load model from URL
# RESULTS_PATH = None
# MODEL_NAME = None
# MODEL_URL = "https://your-platform.com/model.zip"

### Data input configuration

In [ ]:
# Choose ONE of the following options:

# Option 1: File-based inference (memory efficient for large datasets)
DATA_PATH = project_root / "data" / "scXpand_counts_with_expansion_for_model_08_12_2024.h5ad"

# Option 2: In-memory inference (faster for smaller datasets)
# Uncomment and modify one of these lines, then set DATA_PATH = None:
# adata = ad.read_h5ad("your_data.h5ad")  # Load from file into memory
# adata = your_existing_adata_object      # Use existing AnnData object
adata = None  # Set this to your AnnData object for in-memory mode

# If using in-memory mode, set DATA_PATH = None
# DATA_PATH = None

# Optional: Use subset of data for evaluation
# If SPLIT_PATH is not None, only the patient IDs in the subset will be used for evaluation
SPLIT_PATH = project_root / "results" / "optuna_studies" / "dev_patient_ids.csv"
# Otherwise, the full dataset will be used for evaluation:
# SPLIT_PATH = None  # Use full dataset

### Additional configuration

In [ ]:
# Inference parameters
BATCH_SIZE = 2048
NUM_WORKERS = 4

In [ ]:
# Set save path (if None, results will not be saved)
SAVE_PATH = None
# SAVE_PATH = project_root / "results/inference_results"

if SAVE_PATH:
    SAVE_PATH.mkdir(parents=True, exist_ok=True)

## Load Data

The notebook automatically detects your data input mode:
- **File mode**: If `DATA_PATH` is provided and `adata` is None
- **Memory mode**: If `adata` is provided and `DATA_PATH` is None

In [ ]:
# Load data based on configuration
if adata is None and DATA_PATH is not None:
    # File-based mode: load as backed for memory efficiency
    print(f"Loading data from file: {DATA_PATH}")
    adata = ad.read_h5ad(DATA_PATH, backed="r")
    print(f"Loaded {adata.n_obs} cells, {adata.n_vars} genes (file-backed)")
elif adata is not None:
    # Memory mode: adata already provided in configuration
    print(f"Using provided AnnData object: {adata.n_obs} cells, {adata.n_vars} genes")
else:
    raise ValueError("Must provide either DATA_PATH or adata in configuration")

In [ ]:
# Get subset of data for evaluation (if specified)

if SPLIT_PATH:
    with open(SPLIT_PATH) as f:
        dev_patient_ids = [line.strip() for line in f]
    patient_identifiers = get_patient_identifiers(obs_df=adata.obs)
    eval_row_inds = np.where(patient_identifiers.isin(dev_patient_ids))[0]
else:
    eval_row_inds = np.arange(len(adata))

n_cells_eval = len(eval_row_inds)
assert n_cells_eval > 0, "No cells found for evaluation"
print(f"Evaluating on {n_cells_eval} cells ({n_cells_eval / len(adata) * 100:.2f}% of total)")

In [ ]:
# Run inference using the unified scXpand API

# Use the unified inference function - automatically handles all model types!
results = scxpand.run_inference(
    data_path=DATA_PATH,
    adata=adata,
    model_path=RESULTS_PATH,
    model_name=MODEL_NAME,
    model_url=MODEL_URL,
    save_path=SAVE_PATH,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    eval_row_inds=eval_row_inds,
)

# Extract results
y_pred_prob = results.predictions

print(f"Generated predictions for {len(y_pred_prob)} cells")
print(f"Example predictions: {y_pred_prob[:5]}")

## Evaluate Predictions

In [ ]:
if results.has_metrics:
    overall_df, category_df = metrics_dict_to_dataframes(results.metrics, precision=4)

    # Display overall metrics
    if overall_df is not None:
        print("Overall Metrics:")
        display(overall_df)

    # Display category-specific metrics
    if category_df is not None:
        print("\nCategory-Specific Metrics:")
        display(category_df)

## Overall ROC Curve

In [ ]:
if results.has_metrics:
    eval_obs = adata[eval_row_inds].obs
    y_true = extract_is_expanded(eval_obs)

    overall_auroc = plot_roc_curve(
        labels=y_true,
        probs_pred=y_pred_prob,
        show_plot=True,
        plot_save_dir=SAVE_PATH,
        plot_name="roc_curve_overall",
        title="ROC Curve (all cells)",
    )
    print(f"Overall AUROC (all cells): {overall_auroc:.5f}. Plot saved to {SAVE_PATH}")

## Per-Strata AUROC Evaluation

In [ ]:
# Per-strata evaluation
if results.has_metrics:
    strata_cols = ["tissue_type", "imputed_labels"]
    strata_df = eval_obs[strata_cols]
    strata = strata_df.astype(str).agg(" - ".join, axis=1)
    strata.index = eval_obs.index

    unique_strata = strata.unique()
    print(f"Unique strata found: {unique_strata}")

    print(f"Harmonic Average AUROC per Strata: {results.get_harmonic_avg_auroc():.5f}")

In [ ]:
if results.has_metrics:
    auroc_per_strata = plot_roc_curves_per_strata(
        y_true=y_true,
        y_pred_prob=y_pred_prob,
        obs_df=eval_obs,
        strata_columns=strata_cols,
        show_plot=True,
        plot_save_dir=SAVE_PATH,
        max_cols=2,
    )

## Save Predictions

In [ ]:
import pandas as pd


def save_predictions(
    predictions: np.ndarray,
    adata: ad.AnnData,
    save_path: Path,
) -> None:
    if not SAVE_PATH:
        return
    pred_column_name = "expansion_probability"
    predictions_df = pd.DataFrame({"cell_id": adata.obs.index, pred_column_name: predictions})
    csv_path = save_path / "predictions.csv"
    try:
        predictions_df.to_csv(csv_path, index=False)
        print(f"Saved predictions to {csv_path}")
    except Exception:
        print(f"Failed to save predictions to {csv_path}")
        raise


# Note: We need to create a subset of adata that matches the predictions
# Since predictions are already subset to eval_row_inds, we need matching adata subset
eval_adata_subset = adata[eval_row_inds]
save_predictions(predictions=y_pred_prob, adata=eval_adata_subset, save_path=SAVE_PATH)

In [ ]:
# Clean up: close file if it was opened in backed mode
if hasattr(adata, "isbacked") and adata.isbacked:
    adata.file.close()
    print("Closed backed AnnData file")